In [2]:
import pandas as pd
pd.read_excel('07-books-input-1.xlsx',sheet_name=None)

{'genres':    book  Literary  Sci-Fi  Romance  Thriller
 0     1       1.0     NaN      NaN       NaN
 1     2       NaN     1.0      NaN       1.0
 2     3       NaN     NaN      1.0       1.0
 3     4       1.0     NaN      1.0       NaN
 4     5       1.0     NaN      NaN       NaN
 5     6       NaN     NaN      1.0       NaN
 6     7       NaN     1.0      NaN       NaN
 7     8       NaN     NaN      NaN       1.0
 8     9       1.0     1.0      NaN       NaN
 9    10       NaN     NaN      1.0       NaN,
 'requirements':       genre  required
 0  Literary         2
 1    Sci-Fi         2
 2   Romance         2
 3  Thriller         2}

In [3]:
pd.read_excel('07-books-input-1.xlsx',sheet_name='genres')

,book,Literary,Sci-Fi,Romance,Thriller
0,1,1.0,NaN,NaN,NaN
1,2,NaN,1.0,NaN,1.0
2,3,NaN,NaN,1.0,1.0
3,4,1.0,NaN,1.0,NaN
4,5,1.0,NaN,NaN,NaN
5,6,NaN,NaN,1.0,NaN
6,7,NaN,1.0,NaN,NaN
7,8,NaN,NaN,NaN,1.0
8,9,1.0,1.0,NaN,NaN
9,10,NaN,NaN,1.0,NaN


In [4]:
pd.read_excel('07-books-input-1.xlsx',sheet_name='genres', index_col = 0)

,Literary,Sci-Fi,Romance,Thriller
book,,,,
1,1.0,NaN,NaN,NaN
2,NaN,1.0,NaN,1.0
3,NaN,NaN,1.0,1.0
4,1.0,NaN,1.0,NaN
5,1.0,NaN,NaN,NaN
6,NaN,NaN,1.0,NaN
7,NaN,1.0,NaN,NaN
8,NaN,NaN,NaN,1.0
9,1.0,1.0,NaN,NaN


In [7]:
table = pd.read_excel('07-books-input-1.xlsx',sheet_name='genres', index_col = 0).fillna(0)
table

,Literary,Sci-Fi,Romance,Thriller
book,,,,
1,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,1.0
3,0.0,0.0,1.0,1.0
4,1.0,0.0,1.0,0.0
5,1.0,0.0,0.0,0.0
6,0.0,0.0,1.0,0.0
7,0.0,1.0,0.0,0.0
8,0.0,0.0,0.0,1.0
9,1.0,1.0,0.0,0.0


In [9]:
pd.read_excel('07-books-input-1.xlsx',sheet_name='requirements',index_col=0)

,required
genre,
Literary,2
Sci-Fi,2
Romance,2
Thriller,2


In [8]:
q=pd.read_excel('07-books-input-1.xlsx',sheet_name='requirements',index_col=0)['required']
q

genre
Literary    2
Sci-Fi      2
Romance     2
Thriller    2
Name: required, dtype: int64

In [13]:
table.to_excel('12-genres.xlsx')

In [14]:
table.to_excel('12-genres-noindex.xlsx', index = False)

In [12]:
writer=pd.ExcelWriter('12-books.xlsx')
table.to_excel(writer,sheet_name='genres')
q.to_excel(writer,sheet_name='requirements')
writer.close()

In [15]:
selection=pd.DataFrame([2,3,4,9],columns=['books'])
selection

,books
0,2
1,3
2,4
3,9


In [16]:
objective=pd.DataFrame([4],columns=['books_needed'])
objective

,books_needed
0,4


In [17]:
writer=pd.ExcelWriter('12-books-output-1.xlsx')
selection.to_excel(writer,sheet_name='optimal_decision')
objective.to_excel(writer,sheet_name='objective',index=False)
writer.close()

In [18]:
table.to_csv('12-genres.csv')

In [19]:
pd.read_csv('12-genres.csv',index_col=0)

,Literary,Sci-Fi,Romance,Thriller
book,,,,
1,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,1.0
3,0.0,0.0,1.0,1.0
4,1.0,0.0,1.0,0.0
5,1.0,0.0,0.0,0.0
6,0.0,0.0,1.0,0.0
7,0.0,1.0,0.0,0.0
8,0.0,0.0,0.0,1.0
9,1.0,1.0,0.0,0.0


In [44]:
import pandas as pd
from gurobipy import Model, GRB

inputFile = '07-books-input-1.xlsx'
outputFile = '12-books-input-1.xlsx'

table = pd.read_excel(inputFile, sheet_name = 'genres', index_col = 0).fillna(0)
a = table
B = a.index
G = a.columns
q = pd.read_excel(inputFile, sheet_name='requirements', index_col = 0)['required']


mod = Model()
x = mod.addVars(B, vtype = GRB.BINARY, name = 'x')
mod.setObjective(sum(x[b] for b in B))
for g in G:
    mod.addConstr(sum(a.loc[b, g]*x[b] for b in B) >= q[g])
mod.setParam('OutputFlag', False)
mod.optimize()
# mod.objval
writer = pd.ExcelWriter(outputFile)
pd.DataFrame([mod.objval], columns = ['books_needed'])\
    .to_excel(writer, sheet_name = 'objective', index = False)
pd.DataFrame([b for b in B if x[b].x], columns = ['books'])\
    .to_excel(writer, sheet_name = 'optimal_decision')
writer.close()